In [41]:
#importing lib plus setup reading for csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer #TfIdfVectorizer from sklearner
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#setting up the dataets
'''
    df1 contains 
    movieID (unique for each movie)
    cast (each actors/actress including supporting artists)
    crew (including te directors, writer, editors, etc)
'''
df1 = pd.read_csv('./tmdb-movie-metadata/tmdb_5000_credits.csv')

'''
    df2 contains 
    budget, genre, hompage, 
    id, keywords, original_language,
    original_title, overview, popularity
    prouctioncompanies, production_counrties,
    release_date, revenue, runtime, status, tagline
    title, vote_avg, vote_count
'''
df2 = pd.read_csv('./tmdb-movie-metadata/tmdb_5000_movies.csv')

In [11]:
#mergig the two datsets based off of id
df1.columns = ['id','title','cast','crew']
df2= df2.merge(df1,on='id')

In [12]:
#test to see merge
df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,vote_count,title_y,cast_x,crew_x,tittle,cast_y,crew_y,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
#compute pairwise similarity sores for all movies based o their plot desciptions and recomendovies base on the similarity score
#look at movie overview
df2['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [14]:
#defining tf-idf vectorizer object 
tfidf = TfidfVectorizer(stop_words='english') #removesall hte 'filler' words from the list 

#fill empty columns with empty strings
df2['overview'] = df2['overview'].fillna('')

#build the matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['overview'])

#output the shape for the matrix
tfidf_matrix.shape #(movies, words) ratio

(4803, 20978)

In [15]:
#calculate the cosine similarity score
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
#construct reversemap of indices and movie titiles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [17]:
#function takes in title and outputs similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    #get the idex of the movie
    idx = indices[title]
    
    #get the pairwise similarty scores of al movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #sort the films on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #retrieve the top ten scores 
    sim_scores = sim_scores[1:11]
    
    #get the movies
    movie_indices = [i[0] for i in sim_scores]
    
    #return 10 most similar
    return df2['title'].iloc[movie_indices]

In [18]:
get_recommendations('Avatar')

3604                       Apollo 18
2130                    The American
634                       The Matrix
1341            The Inhabited Island
529                 Tears of the Sun
1610                           Hanna
311     The Adventures of Pluto Nash
847                         Semi-Pro
775                        Supernova
2628             Blood and Chocolate
Name: title, dtype: object

In [19]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [20]:
get_recommendations('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title, dtype: object

## Finding Similarities based off of features

In [22]:
#parsing the stringified features into their corresponding objects
features = ['cast', 'crew', 'keywords', 'genres']

#apply string literals for all features
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [23]:
# extract information from features
#get diretor name and crew feature 

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    #if the director is not found return none
    return np.nan

In [24]:
#return list of top elments 
def get_list(x):
    
    if isinstance(x, list):
        name = [i['name'] for i in x]
        
        #return top there from the list
        if len(name) > 3:
            return name[:3]

        return name
    
    #return empty list if no info
    return []

In [28]:
#create a new director/crew feature 
df2['director'] = df2['crew'].apply(get_director)

#features for m
feature = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [30]:
#pinting features for the first three films
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [35]:
#cleaning data/ strip unnecessary white space and lower all char
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #check if director exists
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [36]:
# apply clean data to all features
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [38]:
#creating a metadata soup to feed to the vectorizer
def create_soup(x):
        return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [40]:
#creating a count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [42]:
# calculate the cosine similarity matrix
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [44]:
#reset the inde of the dataframe and onstruct reverse mapping 
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

## Results

In [45]:
get_recommendations('Avatar', cosine_sim2)

206                         Clash of the Titans
71        The Mummy: Tomb of the Dragon Emperor
786                           The Monkey King 2
103                   The Sorcerer's Apprentice
131                                     G-Force
215      Fantastic 4: Rise of the Silver Surfer
466                            The Time Machine
715                           The Scorpion King
1      Pirates of the Caribbean: At World's End
5                                  Spider-Man 3
Name: title, dtype: object

In [46]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

In [48]:
get_recommendations('The Avengers', cosine_sim2)

7                  Avengers: Age of Ultron
26              Captain America: Civil War
79                              Iron Man 2
169     Captain America: The First Avenger
174                    The Incredible Hulk
85     Captain America: The Winter Soldier
31                              Iron Man 3
33                   X-Men: The Last Stand
68                                Iron Man
94                 Guardians of the Galaxy
Name: title, dtype: object

In [50]:
get_recommendations('The Shawshank Redemption', cosine_sim2)

2609                                         Dark Blue
2818                                           Witness
4638                          Amidst the Devil's Wings
2926                                  Dead Man Walking
1283                               The Hudsucker Proxy
1663                       Once Upon a Time in America
1847                                        GoodFellas
1946    The Bad Lieutenant: Port of Call - New Orleans
2582                        The Place Beyond the Pines
3242                              Escape from Alcatraz
Name: title, dtype: object

In [52]:
get_recommendations('The Silence of the Lambs', cosine_sim2)

497          Red Dragon
970     Hannibal Rising
3257    American Psycho
2072    Freedom Writers
135         The Wolfman
717        Jack Reacher
918          Inside Man
924        The Ring Two
1010         Panic Room
2635        Funny Games
Name: title, dtype: object